In [ ]:
from transformers import pipeline

In [ ]:
import pandas as pd

In [ ]:
import re

In [ ]:
import csv

In [ ]:
sentimental_pipeline = pipeline("sentiment-analysis")

In [ ]:
df_tweets =pd.read_csv('SYMCA2_tweetonly.csv')

In [ ]:
data = df_tweets['Tweet'].tolist()

In [ ]:
sentimental_pipeline(data)
for item in data:
    item_type = type(item)
    print(f"Data: {item}, Type: {item_type}")

In [ ]:
def clean_text(text):
    # Remove URLs
    text = re.sub(r'http\S+', '', text)
    # Remove special characters and non-alphanumeric characters
    text = re.sub(r'[^\w\s]', '', text)
    
    # Convert to lowercase
    text = text.lower()
    
    return text
    # Clean the 'value' column using the clean_text function
df_tweets['Tweet'] = df_tweets['Tweet'].apply(clean_text)

print(df_tweets)

data = df_tweets['Tweet'].tolist()
#data = ["I love you", "I hate you"]
sentimental_pipeline(data)

    

In [ ]:
import pandas as pd
import re
from textblob import TextBlob  # Assuming you're using TextBlob for sentiment analysis

def clean_text(text):
    # Remove URLs
    text = re.sub(r'http\S+', '', text)
    # Remove special characters and non-alphanumeric characters
    text = re.sub(r'[^\w\s]', '', text)
    
    # Convert to lowercase
    text = text.lower()
    
    return text

# Create a sample DataFrame
data = {
    'Tweet': [
        "I love this beautiful day! 😊",
        "Feeling sad about recent events. 😢",
        "Just hanging out with friends.",
        "Check out this cool link: http://example.com"
    ]
}
df_tweets = pd.DataFrame(data)
df_tweets = df_tweets[df_tweets['Handle'] != '@SouthYorksMCA']

# Clean the 'Tweet' column using the clean_text function
df_tweets['CleanedTweet'] = df_tweets['Tweet'].apply(clean_text)

# Define a sentiment analysis function
def analyze_sentiment(text):
    blob = TextBlob(text)
    sentiment = blob.sentiment.polarity
    return sentiment

# Apply sentiment analysis to the cleaned tweets
df_tweets['Sentiment'] = df_tweets['CleanedTweet'].apply(analyze_sentiment)

# Map sentiment polarity to sentiment labels
df_tweets['SentimentLabel'] = df_tweets['Sentiment'].apply(lambda x: 'Positive' if x > 0 else 'Negative' if x < 0 else 'Neutral')

# Create a new DataFrame with 'Tweet' and 'SentimentLabel'
df_sentiment = df_tweets[['Tweet', 'SentimentLabel', 'Sentiment']]

# Print the original DataFrame with sentiment analysis results
print(df_tweets)

# Print the DataFrame with tweets and their sentiment labels
print(df_sentiment)

df_sentiment.to_csv("output.csv", encoding='utf-8',index=False)


In [ ]:
pip install TextBlob

In [ ]:
import pandas as pd
import re
from transformers import pipeline

def clean_text(text):
    # Remove URLs
    text = re.sub(r'http\S+', '', text)
    # Remove special characters and non-alphanumeric characters
    text = re.sub(r'[^\w\s]', '', text)
    
    # Convert to lowercase
    text = text.lower()
    
    return text

# Read the parquet file into a DataFrame

file_path_SYMCA ='./Dataset/SYMCA1.parquet'

df_tweets = pd.read_parquet(file_path_SYMCA)
df_tweets = df_tweets[df_tweets['Handle'] != '@SouthYorksMCA']


# Clean the 'Tweet' column using the clean_text function
df_tweets['CleanedTweet'] = df_tweets['Tweet'].apply(clean_text)

# Load the specific sentiment analysis model pipeline
specific_model = pipeline(model="finiteautomata/bertweet-base-sentiment-analysis")

# Apply sentiment analysis to the cleaned tweets
df_tweets['Sentiment'] = df_tweets['CleanedTweet'].apply(lambda text: specific_model(text)[0])

# Create a new DataFrame with 'Tweet' and 'Sentiment'
df_sentiment = df_tweets[['Tweet', 'Sentiment']]

# Print the original DataFrame with sentiment analysis results
print(df_tweets)

# Print the DataFrame with original tweets and their sentiment scores
print(df_sentiment)
df_sentiment.to_csv("output.csv", encoding='utf-8',index=False)

In [ ]:
from wordcloud import WordCloud
from wordcloud import STOPWORDS
import matplotlib.pyplot as plt

df_sentiment["Sentiment_Label"] = df_sentiment["Sentiment"].apply(lambda x: x["label"])
# Wordcloud with positive tweets
positive_tweets = df_sentiment['Tweet'][df_sentiment["Sentiment_Label"] == 'POS']
stop_words = ["https", "co", "RT"] + list(STOPWORDS)
positive_wordcloud = WordCloud(max_font_size=50, max_words=100, background_color="white", stopwords = stop_words).generate(str(positive_tweets))
plt.figure()
plt.title("Positive Tweets - Wordcloud")
plt.imshow(positive_wordcloud, interpolation="bilinear")
plt.axis("off")
plt.show()
 
# Wordcloud with negative tweets
negative_tweets = df_sentiment['Tweet'][df_sentiment["Sentiment_Label"] == 'NEG']
stop_words = ["https", "co", "RT"] + list(STOPWORDS)
negative_wordcloud = WordCloud(max_font_size=50, max_words=100, background_color="white", stopwords = stop_words).generate(str(negative_tweets))
plt.figure()
plt.title("Negative Tweets - Wordcloud")
plt.imshow(negative_wordcloud, interpolation="bilinear")
plt.axis("off")
plt.show()


In [ ]:
negative_tweets


In [ ]:
positive_tweets

In [ ]:
df_sentiment['Tweet']

In [ ]:
df_sentiment["Sentiment_Label"] = df_sentiment["Sentiment"].apply(lambda x: x["label"])


In [ ]:
df_sentiment["Sentiment_Label"]

In [ ]:
df_tweets

In [ ]:
import os
import pandas as pd
import re
from transformers import pipeline
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt

def clean_text(text):
    # Remove URLs
    text = re.sub(r'http\S+', '', text)
    # Remove special characters and non-alphanumeric characters
    text = re.sub(r'[^\w\s]', '', text)
    
    # Convert to lowercase
    text = text.lower()
    
    return text

def analyze_sentiments_and_generate_wordcloud(file_path, council_handle, output_folder):
    # Read the parquet file into a DataFrame
    df_tweets = pd.read_parquet(file_path)
    df_tweets = df_tweets[df_tweets['Handle'] != council_handle]

    # Clean the 'Tweet' column using the clean_text function
    df_tweets['CleanedTweet'] = df_tweets['Tweet'].apply(clean_text)

    # Load the specific sentiment analysis model pipeline
    specific_model = pipeline(model="finiteautomata/bertweet-base-sentiment-analysis")

    # Apply sentiment analysis to the cleaned tweets
    df_tweets['Sentiment'] = df_tweets['CleanedTweet'].apply(lambda text: specific_model(text)[0])

    # Create a new DataFrame with 'Tweet' and 'Sentiment'
    df_sentiment = df_tweets[['Tweet', 'Sentiment']]

    # Save sentiment analysis results to CSV
    output_csv = file_path.replace('.parquet', '_sentiment.csv')
    df_sentiment.to_csv(output_csv, encoding='utf-8', index=False)

    # Create wordclouds for positive, negative, and neutral tweets
    df_sentiment["Sentiment_Label"] = df_sentiment["Sentiment"].apply(lambda x: x["label"])
    
    # Wordcloud with positive tweets
    positive_tweets = df_sentiment['Tweet'][df_sentiment["Sentiment_Label"] == 'POS']
    stop_words = ["https", "co", "RT"] + list(STOPWORDS)
    positive_wordcloud = WordCloud(max_font_size=50, max_words=100, background_color="white", stopwords=stop_words).generate(str(positive_tweets))
    plt.figure()
    plt.title(f"Positive Tweets for {council_handle} - Wordcloud")
    plt.imshow(positive_wordcloud, interpolation="bilinear")
    plt.axis("off")
    plt.savefig(os.path.join(output_folder, f"positive_{council_handle}_wordcloud.png"))  # Save the plot
    plt.close()

    # Wordcloud with negative tweets
    negative_tweets = df_sentiment['Tweet'][df_sentiment["Sentiment_Label"] == 'NEG']
    stop_words = ["https", "co", "RT"] + list(STOPWORDS)
    negative_wordcloud = WordCloud(max_font_size=50, max_words=100, background_color="white", stopwords=stop_words).generate(str(negative_tweets))
    plt.figure()
    plt.title(f"Negative Tweets for {council_handle} - Wordcloud")
    plt.imshow(negative_wordcloud, interpolation="bilinear")
    plt.axis("off")
    plt.savefig(os.path.join(output_folder, f"negative_{council_handle}_wordcloud.png"))  # Save the plot
    plt.close()

    # Wordcloud with neutral tweets
    neutral_tweets = df_sentiment['Tweet'][df_sentiment["Sentiment_Label"] == 'NEU']
    stop_words = ["https", "co", "RT"] + list(STOPWORDS)
    neutral_wordcloud = WordCloud(max_font_size=50, max_words=100, background_color="white", stopwords=stop_words).generate(str(neutral_tweets))
    plt.figure()
    plt.title(f"Neutral Tweets for {council_handle} - Wordcloud")
    plt.imshow(neutral_wordcloud, interpolation="bilinear")
    plt.axis("off")
    plt.savefig(os.path.join(output_folder, f"neutral_{council_handle}_wordcloud.png"))  # Save the plot
    plt.close()

# File paths and council handles for all datasets
data = [
    ('./Dataset/SYMCA.parquet', '@SouthYorksMCA'),
    ('./Dataset/EssexCountyCouncil.parquet', '@Essex_CC'),
    ('./Dataset/HamisphereCountyCouncil.parquet', '@hantsconnect'),
    ('./Dataset/SurreyCountyCouncil.parquet', '@SurreyCouncil')
]

# Output folder to save the plots
output_folder = './WordCloudPlots'

# Create the output folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

# Loop through data and apply the analysis code
for file_path, council_handle in data:
    analyze_sentiments_and_generate_wordcloud(file_path, council_handle, output_folder)
